In [0]:
from platform import python_version
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Activation,Dropout,Flatten,Concatenate,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback,EarlyStopping
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import random
import cv2
import os
import json
import pandas as pd
import re

In [0]:
path="drive/My Drive/Deep Learning/"
testpath=path+"test.jsonl"
trainpath=path+"train.jsonl"



 

In [3]:
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data
a=load_jsonl(testpath)
b=load_jsonl(trainpath)


Loaded 10000 records from drive/My Drive/Deep Learning/test.jsonl
Loaded 550152 records from drive/My Drive/Deep Learning/train.jsonl


In [0]:
train_s1=[d['sentence1'] for d in b]
train_s2=[d['sentence2'] for d in b]
train_label=[d['gold_label'] for d in b]
test_s1=[d['sentence1'] for d in a]
test_s2=[d['sentence2'] for d in a]
test_label=[d['gold_label'] for d in a]

KeyError: ignored

In [0]:
df_train = pd.DataFrame(list(zip(train_s1, train_s2,train_label)), 
               columns =['Sentence1', 'Sentence2','Label']) 
df_test = pd.DataFrame(list(zip(test_s1, test_s2,test_label)), 
               columns =['Sentence1', 'Sentence2','Label']) 

df_train=df_train[df_train.Label != '-']
df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)


In [0]:

df_test['Sentence1']=df_test['Sentence1'].str.lower()
df_test['Sentence2']=df_test['Sentence2'].str.lower()
df_train['Sentence1']=df_train['Sentence1'].str.lower()
df_train['Sentence2']=df_train['Sentence2'].str.lower()



In [0]:
df_test['s1'] = df_test['Sentence1'].map(lambda x: re.sub(r'\W+', '', x))
df_test['s2'] = df_test['Sentence2'].map(lambda x: re.sub(r'\W+', '', x))
df_train['s1'] = df_train['Sentence1'].map(lambda x: re.sub(r'\W+', '', x))
df_train['s2'] = df_train['Sentence2'].map(lambda x: re.sub(r'\W+', '', x))
df_test['s1'] = df_test['Sentence1'].str.replace('.', '')
df_test['s2'] = df_test['Sentence2'].str.replace('.', '')
df_train['s1'] = df_train['Sentence1'].str.replace('.', '')
df_train['s2'] = df_train['Sentence2'].str.replace('.', '')


In [0]:
finaltrain=[]
finaltest=[]
finaltrain=(df_train['s1'].tolist())
finaltrain.extend(df_train['s2'].tolist())
finaltest=(df_test['s1'].tolist())
finaltest.extend(df_test['s2'].tolist())
finaltrain.extend(finaltest);




In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
vectors_train = vectorizer.fit_transform(finaltrain)

# dense = vectors.todense()
# denselist = dense.tolist()
# df = pd.DataFrame(denselist, columns=feature_names)


In [0]:
vs1=vectors_train[:len(df_train['s1'])]
vs2=vectors_train[len(df_train['s1']):2*len(df_train['s1'])]
testvs1=vectors_train[2*len(df_train['s1']):2*len(df_train['s1'])+10000]
testvs2=vectors_train[2*len(df_train['s1'])+10000:]


In [81]:
testvs2.shape

(10000, 33241)

In [0]:
from scipy.sparse import coo_matrix, hstack
train_X=hstack([vs1,vs2])
test_X=hstack([testvs1,testvs2])
# traini_labno=np.array(train_labno)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000).fit(train_X, train_labno)

In [0]:
labdict={
    'entailment':0,
    'neutral':1,
    'contradiction':2
    '-':3
}
train_labno=[]
for d in df_train['Label']:
  train_labno.append(labdict[d])



In [0]:
labdict={
    'entailment':0,
    'neutral':1,
    'contradiction':2,
    '-':3
}
test_labno=[]
for d in df_test['Label']:
  test_labno.append(labdict[d])


In [0]:
pred=0
arr=clf.predict(test_X)
for i in range(10000):
  if arr[i]==test_labno[i]:
    pred+=1
  

In [0]:
revlabdict={
    0 : 'entailment',
    1 :'neutral',
    2 :'contradiction',
    3 : '-'
}

In [0]:
f=open('tfidf_op.txt','w')
for i in range(1000):
  f.write(revlabdict[arr[i]])
  f.write('\n')
f.close()